In [146]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [147]:
f1 = open('../Hue/data1.pckl', 'rb')
f0 = open('../Hue/data0.pckl', 'rb')
data1 = pickle.load(f1)
data0 = pickle.load(f0)
f1.close()
f0.close()

In [148]:
test_set_prop = 0.2

In [149]:
temp0 = np.random.choice(range(len(data0)), len(data0))
temp1 = np.random.choice(range(len(data1)), len(data1))

In [267]:
test_human = data0[temp0[:int(test_set_prop*len(data0))]]
train_human = data0[temp0[int(test_set_prop*len(data0)):]]
test_eq = data1[temp1[:int(test_set_prop*len(data1))]]
train_eq = data1[temp1[int(test_set_prop*len(data1)):]]

## Prefiltering functions

In [287]:
def variation_rate(x, tol=0.00001, fixed_div=False):
    x = np.array(x)
    if len(x) < 2:
        raise ValueError('Data length too short.')
    if fixed_div:
        rates = (x[1:] - x[:-1]) / fixed_div
    else:
        x[x>=0] = x[x>=0] + tol
        x[x<0] = x[x<0] - tol
        rates = (x[1:] - x[:-1]) / x[:-1]
    return rates[abs(x[:-1]) > 1E-10]

In [319]:
mean(variation_rate(train_human[6], tol=0.00001, fixed_div=0.01))

-8.9784515764020458

In [280]:
# def get_variation_rate_stats(data):
#     t, x, y, z = get_data(data)
#     rate = data['header']['sampling_rate']
#     if rate != 25:
#         raise ValueError('Default 25 Hz, data ' + str(rate) + ' Hz')
#     variattion_rates = [variation_rate(dt) for dt in [x, y, z]]
#     kurt_xyz = [scipy.stats.kurtosis(dt) for dt in variattion_rates]
#     mean_xyz = [np.mean(dt) for dt in variattion_rates]
#     var_xyz = [np.var(dt) for dt in variattion_rates]
#     return mean_xyz, var_xyz, kurt_xyz

In [281]:
def get_variation_rate_stats(data):
    x, y, z = data[:50], data[50:100], data[100:150]
    variation_rates = [variation_rate(dt) for dt in [x, y, z]]
    kurt_xyz = [scipy.stats.kurtosis(dt) for dt in variation_rates]
    mean_xyz = [np.mean(dt) for dt in variation_rates]
    var_xyz = [np.var(dt) for dt in variation_rates]
    return mean_xyz, var_xyz, kurt_xyz

In [282]:
def cross_time(data, axis):
    data_translation = data - axis
    return sum(data_translation[1:] * data_translation[:-1] < 0)

In [283]:
def get_cross_time(data, axis=[0, 0, 0]):
    x, y, z = data[:50], data[50:100], data[100:150]
    return cross_time(x, axis[0]), cross_time(y, axis[1]), cross_time(z, axis[2])

In [338]:
def get_prop_zeros(data, tol=1.e-12):
    data = np.abs(np.array(data)) < tol
    x, y, z = data[:50], data[50:100], data[100:150]
    return np.mean(x), np.mean(y), np.mean(z)

In [347]:
get_prop_zeros(train_human[7])

(0.88, 1.0, 0.20000000000000001)

In [349]:
get_prop_zeros(train_eq[1])

(0.0, 0.0, 0.0)

In [352]:
max(np.array([max(get_prop_zeros(dt)) for dt in train_eq]))

0.14000000000000001

In [ ]:
max(np.array([max(get_prop_zeros(dt)) for dt in train_eq]))

In [284]:
get_variation_rate_stats(train_human[14])

([-14574.97963795434, -0.4433338215654804, -1.5182474715351542],
 [10196584596.918825, 8.0302053119813515, 1482.840465460974],
 [44.02083333298978, 12.595996922645671, 8.617114752791778])

In [285]:
get_variation_rate_stats(train_eq[22])

([-0.8512983145256684, -4.2389974531934902, 5.2866512087428923],
 [171.34283669971069, 275.46612919089245, 1590.0799831752295],
 [10.031516927671017, 19.491881243600215, 28.496010320958664])

In [199]:
train_human_variation_stat = [get_variation_rate_stats(dt) for dt in train_human]

In [200]:
train_eq_variation_stat = [get_variation_rate_stats(dt) for dt in train_eq]

In [225]:
train_human_crosstime = np.array([get_cross_time(dt) for dt in train_human])
train_human_crosstime_mean = np.array([np.mean(data) for data in train_human_crosstime])
train_human_crosstime_min = np.array([min(data) for data in train_human_crosstime])

In [226]:
train_eq_crosstime = np.array([get_cross_time(dt) for dt in train_eq])
train_eq_crosstime_mean = np.array([np.mean(data) for data in train_eq_crosstime])
train_eq_crosstime_min = np.array([min(data) for data in train_eq_crosstime])

In [238]:
np.mean(train_human_crosstime_mean<10)

0.21812574139976276

In [239]:
np.mean(train_human_crosstime_min<5)

0.21153024911032028